In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import fmuSimulation.gymFMU as ExampleFMU
from fmuSimulation.configReader import configReader
import numpy as np
import sys

config = os.path.abspath('Example.cfg')
cfg = configReader(config)
config = cfg.getMulti(cfg.getSections())
config

No function for resetIO implemented


{'DEFAULT': {},
 'General': {'DEBUG': False, 'LOG': False},
 'FMU': {'fmuPath': '../../matlab/00-Example/TestGrid.fmu',
  'startTime': 0,
  'stopTime': 20,
  'tolerance': 1e-06,
  'createFMU': False,
  'dt': 0.5},
 'Reinforcement Learning': {'fixedValue': '[1, 3, 5]', 'actionInterval': 0.5},
 'Algorithm': {'fixedValue': '[1, 3, 5]'},
 'Environment': {'fixedValue': '[1, 3, 5]'},
 'Reward': {'rewardFactor': 5}}

In [3]:
import gym, ray
from ray.rllib.algorithms import ppo

ray.init()


2022-11-11 14:06:47,565	INFO worker.py:1519 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.9.13
Ray version:,2.1.0
Dashboard:,http://127.0.0.1:8265


In [4]:
agent = ppo.PPO(env=ExampleFMU.gymFMU, config={"env_config": config, "num_workers": 4, "num_gpus": 1})
#for i in range(50):
 #   out = trainer.train()
  #  print(out['episode_reward_max'])

2022-11-11 14:06:49,897	INFO algorithm.py:2303 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-11-11 14:06:49,898	INFO ppo.py:379 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-11-11 14:06:49,900	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=23440) No function for resetIO implemented
(RolloutWorker pid=17604) No function for resetIO implemented
(RolloutWorker pid=28920) No function for resetIO implemented
(RolloutWorker pid=22500) No function for resetIO implemented


(RolloutWorker pid=17604) 2022-11-11 14:06:55,123	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2022-11-11 14:07:00,092	INFO trainable.py:164 -- Trainable.setup took 10.195 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [8]:
N_ITER = 30
s = "{:3d} reward {:6.2f}/{:6.2f}/{:6.2f} len {:6.2f} saved {}"

for n in range(N_ITER):
    result = agent.train()
    file_name = agent.save()

    print(s.format(
        n + 1,
        result["episode_reward_min"],
        result["episode_reward_mean"],
        result["episode_reward_max"],
        result["episode_len_mean"],
        file_name
        ))

  1 reward -241.49/-181.30/-134.84 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000011
  2 reward -215.10/-162.66/-119.73 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000012
  3 reward -198.28/-151.69/-103.50 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000013
  4 reward -199.08/-145.63/-109.58 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000014
  5 reward -186.93/-131.66/-93.88 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000015
  6 reward -164.65/-119.34/-84.56 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000016
  7 reward -155.80/-106.89/-78.05 len  39.00 saved C:\Users\Robert/ray_results\PPO_gymFMU_2022-11-11_14-06-4931bktt4l\checkpoint_000017
  8 reward -137.03/-93.86/-64.88 len  39.00 

In [13]:
for x in range(20):
    print(agent.compute_single_action([10]))


[-0.98804665]
[-0.17939663]
[0.01735783]
[0.07866287]
[-0.7643204]
[-0.43307972]
[0.9011345]
[-0.2728691]
[-0.10905838]
[-0.09465981]
[0.22913456]
[-0.58753395]
[-0.19029713]
[0.13580894]
[0.00219536]
[-0.5944996]
[0.4025879]
[-0.35986042]
[-0.42192078]
[-0.3755989]


In [7]:
ray.shutdown()


In [9]:
import gym.spaces

In [7]:
type(gym.spaces.discrete.Discrete(1))

gym.spaces.discrete.Discrete

In [38]:
gym.spaces.discrete.Discrete(10, start=-2).contains(x.reset()[0])


True

In [33]:
x = ExampleFMU.gymFMU(config)

array([5], dtype=int64)